In [ ]:
# importing all libraries needed
import numpy as np
import pandas as pd
import os
import time as tm

In [ ]:
#Reading the current main directory
main_directory = os.getcwd()

In [ ]:
#Rreading the name of the subfolder
subfolder_with_path = [f.path for f in os.scandir(main_directory) if f.is_dir()]
subfolder_name = [f.name for f in os.scandir(main_directory) if f.is_dir()]

In [ ]:
#Counting number of files on subfolder
list = os.listdir(subfolder_name[1]) 
number_files = len(list)

In [ ]:
# Changing directory to the folder
os.chdir(subfolder_with_path[1])

In [ ]:
#Using a loop to process all files and populate the main data-frame

# Starting process timming
start = tm.time()

# Creating main data-frame
df_files_level = [] #This array will contain data frames from each file as elements

i = 0
while i < number_files:
    
    ##############################################
    # Processing each file
    ##############################################
    # Saving file's name
    file = list[i]
    
    # Reading number and names of sheets
    df_full = pd.read_excel(file, sheet_name = None)
    sheets_names = df_full.keys()
    number_of_sheets = len(sheets_names)
    
    df_sheets_level = [" "]*number_of_sheets #This array will contain data frames from each sheets as elements
    
    # Reading files
    ThreeCT_data = pd.ExcelFile(file)

    # Saving session, drug and gender
    session_drug_gender = file.split('_')
    session = session_drug_gender[0]
    drug = session_drug_gender[1]
    gender = session_drug_gender[2]


    for j in range(0, number_of_sheets):
        
        ##############################################
        # Processing each sheet
        ##############################################
        
        # Reading sheets within files
        df_sheets_level[j] = ThreeCT_data.parse(j,header = None)

        #Dropping the frist 13 rows and the row between the last reading and the "sum"
        rows_to_drop = np.arange(13)
        df_sheets_level[j] = df_sheets_level[j].drop(df_sheets_level[j].index[rows_to_drop])


        #locating empty and iso columns
        e = df_sheets_level[j].loc[13].where(df_sheets_level[j].loc[13].str.contains('empty')).dropna(how='all')
        e.index #To getthe column value
        empty = e.index[0]
        s = df_sheets_level[j].loc[13].where(df_sheets_level[j].loc[13].str.contains('iso')).dropna(how='all')
        s.index #To getthe column value
        iso = s.index[0]

        # Categorizing "duration" labels wether iso or empty
        df_sheets_level[j].iloc[1:2, empty:empty+1] = 'Duration_empty_(s)'
        df_sheets_level[j].iloc[1:2, iso:iso+1] = 'Duration_iso_(s)'

        #Dropping 13th row, now the 0th row
        df_sheets_level[j] =  df_sheets_level[j].drop( df_sheets_level[j].index[0])

        #Creating and assigning new header
        new_header = df_sheets_level[j].iloc[0] #grab the first row for the header
        df_sheets_level[j] = df_sheets_level[j][1:] #take the data less the header row
        df_sheets_level[j].columns = new_header #set the header row as the df header
        
        # Selecting columns of av verlocity and duration for iso and empty
        df_sheets_level[j] = df_sheets_level[j][["Avg. velocity (pixels/s)", "Duration_empty_(s)","Duration_iso_(s)"]]
        df_sheets_level[j].columns=["AVG_VEL_pixels/s", "DURATION_EMPTY_s", "DURATION_ISO_s"]
        df_sheets_level[j].reset_index(drop=True, inplace=True)

        #Removing the Nan row between the last reading a the 'sum'
        df_sheets_level[j] = df_sheets_level[j].dropna(how='all')

        # Creating time-column vector based on measurements' size
        unit = 20/(len(df_sheets_level[j])-1)
        TIME_minutes = np.arange(unit,21,unit)
        TIME_minutes = np.append(TIME_minutes, "SUM")

        # Adding time in minutes' column
        df_sheets_level[j].insert(0, 'TIME_STEP_minutes', TIME_minutes)
        
        #Adding mouse information
        df_sheets_level[j].insert(0,'SESSION_DRUG_GENDER', 
                                  [session + '_'drug + '_' + gender + '_' str(j+1)]*len(df_sheets_level[j]))

        #Summations a diffrences of duration (empty vs iso)
        summ = df_sheets_level[j]["DURATION_EMPTY_s"] + df_sheets_level[j]["DURATION_ISO_s"]
        diff = df_sheets_level[j]["DURATION_ISO_s"] - df_sheets_level[j]['DURATION_EMPTY_s']
        
        # Changing any sum value from zero to 10^20
        summ.loc[summ == 0] = pow(10,20)
        
        # Calculating relative and normalized socialindexes
        df_sheets_level[j]['SOL_IDX_REL'] = 100 * diff/summ
        df_sheets_level[j]['SOL_IDX_NORM'] = 100 * diff/(5*60)

    ################################################################### # Finish of processing sheets
    df_sheets_level = pd.concat(df_sheets_level)
    df_files_level.append(df_sheets_level)

    i += 1
    
################################################################################### # Finish of processing files
appended_data = pd.concat(df_files_level)


In [ ]:
# Going back to the main directory
os.chdir(main_directory)

# Exporting dt to csv
appended_data.to_excel('3CT_data_vetical_format.xlsx',index=False)

end = tm.time()

# Priting running time and files processed
print(i,' Files processed ')
print(' ')
print(' Execution time:', round((end - start),2), 'seconds') 
